In [18]:
import pandas as pd

In [19]:
data = pd.read_csv("train_and_test2.csv")
data.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


In [20]:

X = data.drop(columns=["Passengerid", "2urvived"], axis="columns")
y = data["2urvived"]

X.shape, y.shape

((1309, 26), (1309,))

In [21]:
from sklearn.preprocessing import StandardScaler

scalable = ["Age", "Fare", "Embarked", "Pclass"]

scaler = StandardScaler().fit(X[scalable])

X[scalable] = scaler.transform(X[scalable])
X[:5]

,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,zero.5,...,zero.11,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18
0,-0.581628,-0.503291,0,1,0,0,0,0,0,0,...,0,0,0,0,0.841916,0,0,0.622940,0,0
1,0.658652,0.734744,1,1,0,0,0,0,0,0,...,0,0,0,0,-1.546098,0,0,-1.833116,0,0
2,-0.271558,-0.490240,1,0,0,0,0,0,0,0,...,0,0,0,0,0.841916,0,0,0.622940,0,0
3,0.426099,0.383183,1,1,0,0,0,0,0,0,...,0,0,0,0,-1.546098,0,0,0.622940,0,0
4,0.426099,-0.487824,0,0,0,0,0,0,0,0,...,0,0,0,0,0.841916,0,0,0.622940,0,0


In [22]:
y.value_counts()

0    967
1    342
Name: 2urvived, dtype: int64

In [25]:
1- 967/(342+967)

0.26126814362108475

In [27]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [28]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': [0.01, 0.05, 0.1, 1],
            'kernel': ['rbf', 'poly', 'sigmoid'],
            'shrinking': [True, False],
            'degree': [2, 4]
        },
        'n_iter': 10
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'n_estimators': [100, 200, 300],
            'max_features': ['sqrt', 'log2'],
            'ccp_alpha': [0.01, 0.02],
            'warm_start': [True, False]
        },
        'n_iter': 12
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            "penalty": ['l1', 'l2'],
            'C': [0.01, 0.05, 0.1, 1, 10],
            'fit_intercept': [True, False],
            'warm_start': [True, False]
        },
        'n_iter': 7
    },
    'gaussian_nb': {
        'model': GaussianNB(),
        'params': {
            'var_smoothing': [1e-8, 1e-9, 1e-10],
            'priors': [None, [0.5, 0.5], [0.739, 0.261]]
        },
        'n_iter': 5
    },
    'multi_nb': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0, 1, 5, 10],
            'fit_prior': [True, False]
        },
        'n_iter': 3
    },
    'multi_nb': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'splitter': ['best', 'random'],
            'ccp_alpha': [0.01, 0.015, 0.02],
            'max_features': ['sqrt', 'log2']
        }
        ,
        'n_iter': 10
    }
}

In [29]:
from sklearn.model_selection import RandomizedSearchCV
from tqdm import notebook as nb

scores = []
pb = nb.tqdm(total=len(model_params))

for model_name, model_data in model_params.items():
    search = RandomizedSearchCV(model_data['model'], model_data['params'], cv=5, 
                                        n_iter=model_data['n_iter'], return_train_score=False)
    
    search.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': search.best_score_,
        'best_params': search.best_params_
    })
    
    pb.update(n=1)

  0%|          | 0/5 [00:00<?, ?it/s]

KeyError: 'n_iter'